In [2]:
import os
import mysql.connector
from astropy.io import fits

# Database config 
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '', 
    'database': 'test', 
}

fits_directory = 'C:\\Users\\samds\\Desktop\\OBS\\nas-frontend'

# Create connection
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

# Check for filenames already in the database
cursor.execute('SELECT file_name FROM fits_data')
existing_files = set(row[0] for row in cursor.fetchall())

# Process only the new FITS files
for filename in os.listdir(fits_directory):
    if filename.endswith('.fits') and filename not in existing_files:
        file_path = os.path.join(fits_directory, filename)
        
        with fits.open(file_path) as fits_file:
            header = fits_file[0].header
            
            object_name = header.get('OBJECT', 'N/A')
            instrument_name = header.get('INSTRUME', 'N/A')
            filter_name = header.get('FILTER', 'N/A')
            date_avg = header.get('DATE-AVG', 'N/A')
            
            # Insert the header 
           
            cursor.execute('''
             INSERT INTO fits_data (object_name, file_name, instrument_name, filter, date_avg)
             VALUES (%s, %s, %s, %s, %s)
            ''', (object_name, filename, instrument_name, filter_name, date_avg))
            
            # Print the confirmation 
            print(f"Inserted header data for file: {filename}")

# Commit
conn.commit()

# Close the connection
cursor.close()
conn.close()


Inserted header data for file: 19_46_09_comet2023A3_finder_00008.fits
Inserted header data for file: 20_34_32_HD 14662_00002.fits
Inserted header data for file: 20_59_40_HD_335264_00010.fits
